In [1]:
# Imports
import numpy as np

In [2]:
# Import the tools
from sys import path
path.append('/Users/reid/dev/PythonCode/tm/tools')
# path.append('D:/PythonCode/tm/tools')
from tools import gen_inputs_outputs, logsig, lin, deriv_logsig, deriv_lin

In [3]:
# Define the size of the network
# Order by layers (input, layer1, layer2, ...)
# !!! No output layer yet, wait to gen inputs
layer_sizes = [3, 7]

In [4]:
# Gen inputs, outputs and size of the last layer
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

In [5]:
# Append the output layer size
layer_sizes = np.array(layer_sizes + [S], dtype=int)

In [6]:
# Network shape
layer_sizes

array([3, 7, 3])

In [7]:
#####
# layer_sizes = np.array([1,2,1])

In [8]:
# Inputs
inputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]])

In [9]:
# Outputs
outputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0]])

In [60]:
# Intialize the weights and biases randomly
# Also create the list to store the outputs of each layer
# Store the outptut for each layer
# Finally create the list of the sensitivities
weights_list = []
biases_list = []
n_list = []
a_list = []
s_list = []

In [61]:
for i in range(1, len(layer_sizes)):
    weights_list.append(np.random.rand(layer_sizes[i], layer_sizes[i-1]))
    biases_list.append(np.random.rand(layer_sizes[i], 1))
    n_list.append(np.empty((layer_sizes[i], 1), dtype=float))
    a_list.append(np.empty((layer_sizes[i], 1), dtype=float))
    s_list.append(np.empty((layer_sizes[i], 1), dtype=float))

In [62]:
#####
'''weights_list = [
    np.array([[-0.27],[-0.41]]),
    np.array([[0.09, -0.17]])
]
biases_list = [
    np.array([[-0.48], [-0.13]]),
    np.array([[0.48]])
]
n_list = [
    np.array([[0], [0]]),
    np.array([[0]])
]
a_list = [
    np.array([[0], [0]]),
    np.array([[0]])
]'''

'weights_list = [\n    np.array([[-0.27],[-0.41]]),\n    np.array([[0.09, -0.17]])\n]\nbiases_list = [\n    np.array([[-0.48], [-0.13]]),\n    np.array([[0.48]])\n]\nn_list = [\n    np.array([[0], [0]]),\n    np.array([[0]])\n]\na_list = [\n    np.array([[0], [0]]),\n    np.array([[0]])\n]'

In [63]:
# Define the transfer functions
transfer_functions = [logsig, lin]

In [64]:
# Define the vectors with the derivatives of the transfer functions
# These need to be converted to dialation matrices
# but this is done after the partial derivatives are calculated
# so that the numerical values can be multiplied by the identity matrix
# makes my life easier
deriv_transfer_functions = [deriv_logsig, deriv_lin]

In [65]:
# Define a function to calculate the output of the network
# while also saving the data
# Assuming that the input is already a Rx1 matrix
def run_network(X):
    # Get the input for the first layer so that
    # no need to use X and can calculate
    # recursively
    n_list[0] = np.matmul(weights_list[0], X) + biases_list[0]
    a_list[0] = transfer_functions[0](n_list[0])

    # Calculate the rest of the outptut
    for i in range(1, len(weights_list)):
        n_list[i] = np.matmul(weights_list[i], a_list[i-1]) + biases_list[i]
        a_list[i] = transfer_functions[i](n_list[i])

In [66]:
run_network(inputs[1].reshape(layer_sizes[0], 1))
####
# run_network(np.array([2], dtype=float).reshape(1, 1))

In [67]:
print(n_list, a_list, sep='\n\n')

[array([[0.64710373],
       [0.68921108],
       [0.77334232],
       [0.72906554],
       [1.80432772],
       [1.40324663],
       [1.10607694]]), array([[2.99650275],
       [2.55570339],
       [3.05383181]])]

[array([[0.6563575 ],
       [0.6657914 ],
       [0.68424346],
       [0.67460018],
       [0.85867493],
       [0.80269857],
       [0.75139701]]), array([[2.99650275],
       [2.55570339],
       [3.05383181]])]


In [68]:
# Calculate the senstivites for the last layer
# Write a function to fill the diagonal of the matrix
def fill_F_dot(layer_num):
    a = np.identity(len(n_list[layer_num]), float)
    np.fill_diagonal(a, deriv_transfer_functions[layer_num](n_list[layer_num]).flatten())
    return a

learning_rate = .01
# Iterate through the inputs and outputs
for i in range(len(inputs)):
    # Run the network
    run_network(inputs[i].reshape(layer_sizes[0], 1))

    # Get the dialation matrix for the last layer
    F_dot = fill_F_dot(-1)
    
    # Calculate the sensitivites for the last layer
    s_list[-1] = -2 * np.matmul(F_dot, outputs[i].reshape(layer_sizes[-1], 1)-a_list[-1])

    # Update the weights and biases for the last layer
    weights_list[-1] += -learning_rate*np.matmul(s_list[-1], a_list[-2].T)
    biases_list[-1] += -learning_rate*s_list[-1]

    # Iterate through the remaining layers backwards
    for i in range(len(n_list)-2, 0, -1):
        s_list[i] = np.matmul(np.matmul(fill_F_dot(i), weights_list[i+1].T), s_list[i+1])
        weights_list[i] += -learning_rate*np.matmul(s_list[i], a_list[i-1].T)
        biases_list[i] += -learning_rate*s_list[i]

In [69]:
print(biases_list)

[array([[0.08318732],
       [0.6225437 ],
       [0.65933107],
       [0.12758467],
       [0.83037111],
       [0.63507649],
       [0.45073752]]), array([[0.4338214],
       [0.1071451],
       [0.4305847]])]


In [70]:
for i in range(len(inputs)):
    run_network(inputs[i].reshape(layer_sizes[0], 1))
    print(outputs[i].reshape(layer_sizes[-1], 1)-a_list[-1])

[[-1.22613753]
 [-1.19846589]
 [-1.36370064]]
[[-1.4132286 ]
 [-1.36589764]
 [-0.56587789]]
[[-1.33496565]
 [-0.27620463]
 [-1.47582177]]
[[-1.47554622]
 [-0.41977342]
 [-0.66111117]]
[[-1.30135037]
 [-1.38440527]
 [-0.53567577]]
[[-1.45648567]
 [-0.51399554]
 [-1.69982243]]
[[-1.37973376]
 [-0.44245657]
 [-0.61450086]]
[[-0.4980077 ]
 [-1.5506066 ]
 [-1.75901901]]
